# Bollinger Bands Strategy

This notebook demonstrates a mean reversion strategy using Bollinger Bands.

**Strategy Logic:**
- Buy when price touches lower band (oversold)
- Sell when price touches upper band (overbought)
- Bollinger Bands = Moving Average ± (Standard Deviation × multiplier)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Any, Dict, List

from simple_backtest import Backtest, BacktestConfig
from simple_backtest.strategy.strategy_base import Strategy
from simple_backtest.visualization.plotter import plot_equity_curve

## 1. Implement Bollinger Bands Strategy

In [ ]:
class BollingerBandsStrategy(Strategy):
    """Bollinger Bands mean reversion strategy."""
    
    def __init__(self, window: int = 20, num_std: float = 2.0, shares: float = 100):
        """Initialize Bollinger Bands strategy.
        
        :param window: Moving average window
        :param num_std: Number of standard deviations for bands
        :param shares: Shares to trade
        """
        super().__init__(name=f"BB_{window}_{num_std}")
        self.window = window
        self.num_std = num_std
        self.shares = shares
    
    def calculate_bollinger_bands(self, prices: pd.Series):
        """Calculate Bollinger Bands.
        
        :return: (middle_band, upper_band, lower_band, current_price)
        """
        if len(prices) < self.window:
            return None, None, None, prices.iloc[-1]
        
        # Calculate middle band (SMA)
        middle_band = prices.tail(self.window).mean()
        
        # Calculate standard deviation
        std = prices.tail(self.window).std()
        
        # Calculate upper and lower bands
        upper_band = middle_band + (self.num_std * std)
        lower_band = middle_band - (self.num_std * std)
        
        current_price = prices.iloc[-1]
        
        return middle_band, upper_band, lower_band, current_price
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Generate signal based on Bollinger Bands."""
        # Calculate Bollinger Bands
        middle, upper, lower, current_price = self.calculate_bollinger_bands(data['Close'])
        
        if middle is None:
            return {"signal": "hold", "size": 0, "order_ids": None}
        
        # Check if we have open positions
        has_position = False
        if trade_history:
            total_bought = sum(t['shares'] for t in trade_history if t['signal'] == 'buy')
            total_sold = sum(t['shares'] for t in trade_history if t['signal'] == 'sell')
            has_position = total_bought > total_sold
        
        # Generate signal
        if current_price <= lower and not has_position:
            # Price at or below lower band - oversold, buy
            return {"signal": "buy", "size": self.shares, "order_ids": None}
        elif current_price >= upper and has_position:
            # Price at or above upper band - overbought, sell
            return {"signal": "sell", "size": self.shares, "order_ids": None}
        else:
            return {"signal": "hold", "size": 0, "order_ids": None}

## 2. Download Market Data

In [ ]:
# Download Microsoft data
ticker = "MSFT"
start_date = "2020-01-01"
end_date = "2023-12-31"

data = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {len(data)} rows of {ticker} data")
data.head()

## 3. Visualize Bollinger Bands

In [ ]:
import matplotlib.pyplot as plt

# Calculate Bollinger Bands for visualization
window = 20
num_std = 2.0

data['Middle_Band'] = data['Close'].rolling(window=window).mean()
data['Std'] = data['Close'].rolling(window=window).std()
data['Upper_Band'] = data['Middle_Band'] + (num_std * data['Std'])
data['Lower_Band'] = data['Middle_Band'] - (num_std * data['Std'])

# Plot recent period
recent_data = data.tail(200)

plt.figure(figsize=(14, 7))
plt.plot(recent_data.index, recent_data['Close'], label='Close Price', linewidth=2)
plt.plot(recent_data.index, recent_data['Middle_Band'], label='Middle Band (20 SMA)', linestyle='--', alpha=0.7)
plt.plot(recent_data.index, recent_data['Upper_Band'], label='Upper Band', linestyle='--', color='red', alpha=0.7)
plt.plot(recent_data.index, recent_data['Lower_Band'], label='Lower Band', linestyle='--', color='green', alpha=0.7)
plt.fill_between(recent_data.index, recent_data['Upper_Band'], recent_data['Lower_Band'], alpha=0.1, color='gray')

plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title(f'{ticker} Price with Bollinger Bands (20, 2)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Run Backtest with Default Parameters

In [ ]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,
    execution_price="open",
    risk_free_rate=0.02,
)

# Create and run Bollinger Bands strategy
bb_strategy = BollingerBandsStrategy(window=20, num_std=2.0, shares=50)

backtest = Backtest(data=data, config=config)
results = backtest.run([bb_strategy])

# Display metrics
print("\n=== Bollinger Bands Strategy Results ===")
for key, value in results[bb_strategy.get_name()]['metrics'].items():
    print(f"{key:25s}: {value:>12.2f}")

In [ ]:
# Plot equity curve
fig = plot_equity_curve(results)
fig.show()

## 5. Parameter Optimization

Test different window sizes and standard deviation multipliers.

In [ ]:
# Test different parameters
strategies = []

# Test different windows
for window in [10, 20, 30]:
    strategies.append(BollingerBandsStrategy(window=window, num_std=2.0, shares=50))

# Test different standard deviations
for num_std in [1.5, 2.0, 2.5]:
    if num_std != 2.0:  # Avoid duplicate
        strategies.append(BollingerBandsStrategy(window=20, num_std=num_std, shares=50))

# Run all strategies
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

# Compare results
comparison_df = pd.DataFrame({
    name: res['metrics'] 
    for name, res in results.items() 
    if name != 'benchmark'
}).T

print("\n=== Bollinger Bands Parameter Comparison ===")
comparison_df[['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate', 'total_trades']].sort_values('sharpe_ratio', ascending=False)

In [ ]:
# Plot all equity curves
fig = plot_equity_curve(results)
fig.show()

## 6. Analyze Trade Timing

In [ ]:
# Get best performing strategy
best_strategy = comparison_df['sharpe_ratio'].idxmax()
trades = results[best_strategy]['trade_history']

# Convert to DataFrame
trades_df = pd.DataFrame(trades)
trades_df['timestamp'] = pd.to_datetime(trades_df['timestamp'])

# Plot trades on price chart
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price', alpha=0.7)
plt.plot(data.index, data['Upper_Band'], 'r--', alpha=0.5, label='Upper Band')
plt.plot(data.index, data['Lower_Band'], 'g--', alpha=0.5, label='Lower Band')

# Mark buy and sell signals
buys = trades_df[trades_df['signal'] == 'buy']
sells = trades_df[trades_df['signal'] == 'sell']

plt.scatter(buys['timestamp'], buys['price'], color='green', marker='^', s=100, label='Buy', zorder=5)
plt.scatter(sells['timestamp'], sells['price'], color='red', marker='v', s=100, label='Sell', zorder=5)

plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title(f'{ticker} - Trade Entry/Exit Points - {best_strategy}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nBest Strategy: {best_strategy}")
print(f"Total Trades: {len(trades_df)}")

## 7. Risk Analysis

In [ ]:
# Analyze returns distribution
portfolio_values = results[best_strategy]['portfolio_values']
returns = results[best_strategy]['returns']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Returns distribution
axes[0, 0].hist(returns * 100, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Daily Returns (%)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Returns Distribution')
axes[0, 0].grid(True, alpha=0.3)

# Drawdown
cumulative_max = portfolio_values.cummax()
drawdown = (portfolio_values - cumulative_max) / cumulative_max * 100
axes[0, 1].fill_between(drawdown.index, drawdown, 0, alpha=0.3, color='red')
axes[0, 1].plot(drawdown.index, drawdown, color='red')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Drawdown (%)')
axes[0, 1].set_title('Drawdown Over Time')
axes[0, 1].grid(True, alpha=0.3)

# Monthly returns heatmap (simplified)
monthly_returns = returns.resample('ME').sum() * 100
axes[1, 0].bar(range(len(monthly_returns)), monthly_returns.values)
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Monthly Return (%)')
axes[1, 0].set_title('Monthly Returns')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Rolling Sharpe ratio
rolling_sharpe = returns.rolling(window=60).mean() / returns.rolling(window=60).std() * np.sqrt(252)
axes[1, 1].plot(rolling_sharpe.index, rolling_sharpe)
axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Sharpe Ratio')
axes[1, 1].set_title('60-Day Rolling Sharpe Ratio')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()